In [ ]:
''' required dependencies
matplotlib==3.10.0
numpy==1.26.4
pandas==2.2.3
roifile==2025.2.20
scipy==1.15.2
tifffile==2024.9.20
tqdm==4.67.1
skimage

In [ ]:
# Cell 1 — Imports & helper functions

import os
import numpy as np
import pandas as pd
import tifffile as tiff  # for reading 3D TIFF stacks :contentReference[oaicite:0]{index=0}
import roifile
def read_roi_file(path):
    return roifile.roiread(path)
from skimage.measure import label, regionprops_table  # for component counting :contentReference[oaicite:2]{index=2}
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed  # 3D watershed :contentReference[oaicite:3]{index=3}

# Convert a single ImageJ .roi into a 2D boolean mask of shape (Y,X)
def roi_to_mask(roi_path, shape_xy):
    roi_obj = read_roi_file(roi_path)
    
    # Handle both zipped and single ROI cases
    if isinstance(roi_obj, dict):
        roi = next(iter(roi_obj.values()))
    else:
        roi = roi_obj

    coords = roi.coordinates()
  # list of (x,y)
    if coords is None:
        raise ValueError("ROI has no coordinates.")

    mask = np.zeros(shape_xy, dtype=bool)
    xs, ys = zip(*coords)
    
    # fill polygon:
    from matplotlib.path import Path
    poly = Path(np.column_stack((xs, ys)))
    yy, xx = np.mgrid[:shape_xy[0], :shape_xy[1]]
    pts = np.column_stack((xx.ravel(), yy.ravel()))
    mask_flat = poly.contains_points(pts)
    mask = mask_flat.reshape(shape_xy)
    return mask

# Count & measure objects in a binary volume, optionally applying watershed
def count_objects(seg_binary, min_size, max_size, do_watershed=False):
    if do_watershed:
        # distance transform
        dist = ndi.distance_transform_edt(seg_binary)
        peaks = peak_local_max(dist, footprint=np.ones((3, 3, 3)), labels=seg_binary)
        marker_mask = np.zeros_like(dist, dtype=bool)
        marker_mask[tuple(peaks.T)] = True
        markers, _ = ndi.label(marker_mask)
        seg_binary = watershed(-dist, markers, mask=seg_binary)

    # label & filter by size
    labeled = label(seg_binary)
    props = regionprops_table(
        labeled,
        properties=('label', 'area', 'centroid')
    )
    df = pd.DataFrame(props)
    df = df[(df['area'] >= min_size) & (df['area'] <= max_size)]
    count = len(df)

    # centroids: extract and combine centroid components
    centroid_cols = [col for col in df.columns if col.startswith('centroid-')]
    centroids = list(zip(*(df[col] for col in centroid_cols)))

    return count, centroids

In [ ]:
# Cell 2 — Main processing loop

# USER PARAMETERS (edit as needed)
input_dir   = r"D:\PSM\SST_images\PV2\best_segmentation"      # your segmentation .tif folder
roi_dir     = r"D:\PSM\SST_images\PV2\avg\Processed_ROIs"       # where your .roi files live
output_csv  = r"D:\PSM\SST_images\PV2\csvs.csv"   # final CSV path
output_csv_dir = r"D:\PSM\SST_images\PV2\csvs"   # statistics CSV path
pixel_size = 1551.52 / 1024     # ~1.5151 microns/pixel
min_size    = 80      # voxels
max_size    = 18874368
threshold   = 1       # only for naming consistency

# Prepare output DataFrame
columns = [
    "Image_Name","Full_ROI_Area_Microns","Full_Object_Count",
    "Oriens_ROI_Area_Microns","Oriens_Object_Count",
    "Pyrimidale_ROI_Area_Microns","Pyrimidale_Object_Count",
    "Radiatum_ROI_Area_Microns","Radiatum_Object_Count",
    "Centroid_XY"
]
results = []

In [ ]:
# iterate files
for fname in sorted(os.listdir(input_dir)):
    if not fname.endswith(".tif"): continue
    parts    = fname.split("_")
    base     = "_".join(parts[:-2])
    seg_path = os.path.join(input_dir, fname)
    stack    = tiff.imread(seg_path)  # shape (Z,Y,X) :contentReference[oaicite:4]{index=4}

    # read ROIs
    roi_full     = os.path.join(roi_dir, f"{base}_full.roi")
    roi_oriens   = os.path.join(roi_dir, f"{base}_s_oriens.roi")
    roi_pyr      = os.path.join(roi_dir, f"{base}_s_pyrimidale.roi")
    roi_rad      = os.path.join(roi_dir, f"{base}_s_radiatum.roi")
    if not all(os.path.exists(p) for p in [roi_full,roi_oriens,roi_pyr,roi_rad]):
        print(f"Skipping {fname}: missing ROI")
        continue

    # build 3D masks by stamping the 2D ROI across all Z
    _, H, W = stack.shape
    mask_full   = roi_to_mask(roi_full, (H,W))
    mask_oriens = roi_to_mask(roi_oriens, (H,W))
    mask_pyr    = roi_to_mask(roi_pyr,  (H,W))
    mask_rad    = roi_to_mask(roi_rad,  (H,W))
    masks3d = {
        "Full":   np.broadcast_to(mask_full,   stack.shape),
        "Oriens": np.broadcast_to(mask_oriens, stack.shape),
        "Pyrimidale": np.broadcast_to(mask_pyr, stack.shape),
        "Radiatum":   np.broadcast_to(mask_rad, stack.shape)
    }

    row = [base]
    centroids_all = []

    # process full & subdivisions
    for region in ["Full","Oriens","Pyrimidale","Radiatum"]:
        m3d = masks3d[region]
        # ROI area:
        area_px = m3d.sum()
        area_um = area_px * (pixel_size**2)
        # object counting
        seg_bin = (stack>0) & m3d
        count, cents = count_objects(seg_bin, min_size, max_size, do_watershed=True)
        # record
        row.extend([area_um, count])
        if region=="Full":
            centroids_all = cents

    # format centroids
    cent_str = "; ".join(
        "({:.2f},{:.2f})".format(c[-1]*pixel_size, c[-2]*pixel_size)  # x, y from (z, y, x) or (y, x)
        for c in centroids_all
    )
    row.append(cent_str)

    results.append(row)
    print(f"Done {fname}")

# save
df = pd.DataFrame(results, columns=columns)
df.to_csv(output_csv, index=False)
print("Processing complete. Results at:", output_csv)

In [ ]:
# Cell 3: Restrict objects to Full ROI + Watershed + Volume in microns³ + Save per-image object table

import os
import numpy as np
import pandas as pd
from tifffile import imread, TiffFile
from skimage.measure import label, regionprops_table
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from tqdm import tqdm

# --- Output directory for FIJI-style object tables ---
output_dir = os.path.join(output_csv_dir)
os.makedirs(output_dir, exist_ok=True)

# --- Determine Z spacing from metadata of first .tif file ---
tif_files = sorted(f for f in os.listdir(input_dir) if f.endswith(".tif"))
if not tif_files:
    raise FileNotFoundError("No .tif files found in input_dir")

with TiffFile(os.path.join(input_dir, tif_files[0])) as tif:
    meta = tif.imagej_metadata or {}
    z_spacing_um = float(meta.get("spacing", 1.0))

voxel_size = np.array([z_spacing_um, pixel_size, pixel_size])
voxel_volume_um3 = np.prod(voxel_size)

# --- Threshold for large object splitting ---
cell_dims_um = [30, 18, 15]
cell_volume_um3 = (4/3) * np.pi * np.prod(np.array(cell_dims_um)/2)
large_voxel_threshold = int(np.ceil(cell_volume_um3 / voxel_volume_um3))

# --- Watershed helpers ---
def adaptive_watershed(mask, voxel_size, initial_sigma=1.0, max_attempts=6):
    dist = ndi.distance_transform_edt(mask, sampling=voxel_size)
    fallback = mask.astype(np.uint16)
    for attempt in range(max_attempts):
        sigma = max(0.5, initial_sigma - 0.5 * attempt)
        footprint = np.ones((3 + 2 * attempt,) * 3)
        min_dist = max(1, 6 - attempt)
        smoothed = ndi.gaussian_filter(dist, sigma=sigma)
        peaks = peak_local_max(smoothed, labels=mask, footprint=footprint, min_distance=min_dist)
        if peaks.size == 0:
            continue
        markers = np.zeros_like(mask, dtype=bool)
        markers[tuple(peaks.T)] = True
        markers_label, _ = ndi.label(markers)
        ws = watershed(-smoothed, markers_label, mask=mask)
        if np.max(ws) >= 2:
            return ws
    return fallback

def selective_watershed(seg_binary, voxel_size, size_threshold):
    lbl = label(seg_binary)
    out = np.zeros_like(lbl, dtype=np.uint16)
    idx = 1
    for region in np.unique(lbl):
        if region == 0:
            continue
        mask = (lbl == region)
        if np.sum(mask) <= size_threshold:
            out[mask] = idx
            idx += 1
        else:
            ws = adaptive_watershed(mask, voxel_size)
            for part in np.unique(ws):
                if part == 0:
                    continue
                out[ws == part] = idx
                idx += 1
    return out

# --- Process each image ---
for fname in tqdm(tif_files, desc="Processing images"):
    if not fname.endswith(".tif"):
        continue

    parts = fname.split("_")
    base = "_".join(parts[:-2])
    img_path = os.path.join(input_dir, fname)
    stack = imread(img_path)

    # Load Full ROI and apply it as a 3D mask
    roi_path = os.path.join(roi_dir, base + "_full.roi")
    if not os.path.exists(roi_path):
        print("Skipping {}: missing full ROI".format(fname))
        continue

    _, H, W = stack.shape
    mask_full_2d = roi_to_mask(roi_path, (H, W))
    mask_full_3d = np.broadcast_to(mask_full_2d, stack.shape)

    # Apply ROI restriction to segmentation
    seg_bin = (stack > 0) & mask_full_3d

    # Apply selective watershed
    ws_labels = selective_watershed(seg_bin, voxel_size, large_voxel_threshold)

    # Extract object properties
    props = regionprops_table(
        ws_labels, properties=['label', 'area', 'centroid', 'bbox']
    )
    df = pd.DataFrame(props).rename(columns={
        'area': 'Volume (µm^3)',
        'centroid-0': 'Z', 'centroid-1': 'Y', 'centroid-2': 'X',
        'bbox-0': 'BoundingBoxZ0', 'bbox-1': 'BoundingBoxY0', 'bbox-2': 'BoundingBoxX0',
        'bbox-3': 'BoundingBoxZ1', 'bbox-4': 'BoundingBoxY1', 'bbox-5': 'BoundingBoxX1',
    })

    # Save output
    out_file = os.path.join(output_dir, f"Statistics_for_{fname}.csv")
    df.to_csv(out_file, index=False)
    print("Saved:", out_file)

print("✅ Cell D complete — volumes in µm³, detection restricted to Full ROI.")


In [ ]:
'''# Cell 4: Max projection of binary segmentation with centroid markers and ROI outline

import matplotlib.pyplot as plt
from skimage.measure import find_contours

# Confirm required variables exist
assert 'stack' in locals() and 'df' in locals() and 'mask_full_2d' in locals(), "Run Cell D first"

# Max projection of the binary segmentation (objects in white)
proj = np.max(stack, axis=0)

# Convert centroid coordinates back to pixels
df['Y_index'] = (df['Y'] / pixel_size).round().astype(int)
df['X_index'] = (df['X'] / pixel_size).round().astype(int)

# Find ROI outline from the 2D full ROI mask
roi_outline = find_contours(mask_full_2d.astype(float), level=0.5)

# Plot
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(proj, cmap='gray')
ax.set_title(f"{base} - Max Projection with Centroids and ROI")
ax.axis("off")

# Plot centroids
ax.plot(
    df['X_index'], df['Y_index'],
    marker='+', linestyle='none', color='red', markersize=8, markeredgewidth=1.5
)

# Plot ROI outline in yellow
for contour in roi_outline:
    ax.plot(contour[:, 1], contour[:, 0], color='yellow', linewidth=1.5)

plt.tight_layout()
plt.show()'''
